<a href="https://colab.research.google.com/github/isakdiaz/treenet-colab/blob/main/treenet_coreml_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pprint
import json
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint


In [3]:
print(tf.__version__)
print(np.__version__)

2.6.0
1.19.5


## Load Cloud Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install CoreML Tools
### Requires Python 3.5 +

In [5]:
!pip install coremltools

     |████████████████████████████████| 3.4 MB 30.7 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=6c63e5a5103f8dd4c1ecc9027f82d86ec83dd72b78a0b29c5c8d1ad284b1a6d3
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [6]:
import coremltools as ct

## Download dataset to get classes


In [7]:
# Dataset URL
!gdown https://drive.google.com/uc?id=1N-9k7Aa-GRAoiuKgM7ERPn-UkKdgcfIa

Downloading...
From: https://drive.google.com/uc?id=1N-9k7Aa-GRAoiuKgM7ERPn-UkKdgcfIa
To: /content/treenet.zip
308MB [00:02, 115MB/s]


In [ ]:
# Place the plantnet zipped file in a folder called plantnet on your Google Drive directory
# Unzips files from google drive to google colab
!unzip /content/treenet.zip

## Create Class Labels for CoreML Config File

In [12]:
# Get labels from folder names
DATASET_PATH = '/content/treenet'
children= [os.path.join(DATASET_PATH, child) for child in os.listdir(DATASET_PATH)]
files = filter(os.path.isdir, children)
labels = sorted([file.split("/")[-1] for file in files])
num_classes = len(labels)

print(f"Dataset contains {num_classes} classes.")
labels

Dataset contains 6 classes.


['magnolia-grandiflora',
 'nothing-found',
 'phoenix-sylvestris',
 'pinus-clausa',
 'quercus-virginiana',
 'sabal-palmetto']

# Load Model File


In [16]:
# Check if a model exists
!ls -all -h /content/drive/MyDrive/saved_models/treenet

total 20M
drwx------ 2 root root 4.0K Sep 29 14:25 logs
-rw------- 1 root root  20M Sep 29 14:35 treenet_20210929.h5


## Load Keras Model

In [20]:
gdrive_dir = "/content/drive/MyDrive"
model_name = "treenet_20210929"
file_path = "{0}/saved_models/treenet/{1}.h5".format(gdrive_dir, model_name)
tf_model = tf.keras.models.load_model(file_path)

In [15]:
tf_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

# Convert Keras model to CoreML Model


In [17]:
# image_input = ct.ImageType(color_layout="RGB", scale=1/127.0, bias=[-1,-1,-1]) # -1 to 1 input
image_input = ct.ImageType(color_layout="RGB", scale=1/255.0, bias=[0,0,0]) # 0 to 1 input
classifier_config = ct.ClassifierConfig(labels)

# Set input as ImageType so CoreML can automatically resize it using Vision framework
coreml_model = ct.convert(tf_model, inputs=[image_input], classifier_config=classifier_config)

Translating MIL ==> MLModel Ops: 100%|██████████| 751/751 [00:00<00:00, 1374.48 ops/s]


In [18]:
# # Define Spec Function
from coremltools.models.neural_network.builder import _get_nn_spec as get_nn

## Get Spec and check preprocessing
spec = coreml_model.get_spec()
nn = get_nn(spec)
print(nn.preprocessing)

[featureName: "input_2"
scaler {
  channelScale: 0.003921568859368563
}
]


## Save CoreML Model to Google Drive

In [21]:
# !ls -all -hs "$gdrive_dir"
folder_name = gdrive_dir + "/CoreML"
!mkdir -p {folder_name}
coreml_file_path = "{0}/{1}.mlmodel".format(folder_name, model_name)
coreml_model.save(coreml_file_path)
print("Core ML model {} saved in {}".format(model_name, folder_name))

Core ML model treenet_20210929 saved in /content/drive/MyDrive/CoreML
